**Partie 2**

In [11]:
import numpy as np
from matplotlib import pyplot as plt
import cv2 as cv
import glob

# B1 Calibration
import numpy
import cv2
from cv2 import aruco
import pickle
import glob
# ChAruco board variables
CHARUCOBOARD_ROWCOUNT = 6
CHARUCOBOARD_COLCOUNT = 10
ARUCO_DICT = aruco.Dictionary_get(aruco.DICT_5X5_1000)

# Create constants to be passed into OpenCV and Aruco methods
CHARUCO_BOARD = aruco.CharucoBoard_create(
        squaresX=CHARUCOBOARD_COLCOUNT,
        squaresY=CHARUCOBOARD_ROWCOUNT,
        squareLength=0.04,
        markerLength=0.02,
        dictionary=ARUCO_DICT)

# Create the arrays and variables we'll use to store info like corners and IDs from images processed
corners_all = [] # Corners discovered in all images processed
ids_all = [] # Aruco ids corresponding to corners discovered
image_size = None # Determined at runtime
images = glob.glob('C:/Users/yadrina/Downloads/B1/Calibration_B1/Intrinsic/*.tiff')


for iname in images:
    # Open the image
    img = cv2.imread(iname)
    # Grayscale the image
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Find aruco markers in the query image
    corners, ids, _ = aruco.detectMarkers(
            image=gray,
            dictionary=ARUCO_DICT)

    # Outline the aruco markers found in our query image
    img = aruco.drawDetectedMarkers(
            image=img, 
            corners=corners)

    # Get charuco corners and ids from detected aruco markers
    response, charuco_corners, charuco_ids = aruco.interpolateCornersCharuco(
            markerCorners=corners,
            markerIds=ids,
            image=gray,
            board=CHARUCO_BOARD)

    # If a Charuco board was found, let's collect image/corner points
    # Requiring at least 20 squares
    if response > 20:
        # Add these corners and ids to our calibration arrays
        corners_all.append(charuco_corners)
        ids_all.append(charuco_ids)
        
        # Draw the Charuco board we've detected to show our calibrator the board was properly detected
        img = aruco.drawDetectedCornersCharuco(
                image=img,
                charucoCorners=charuco_corners,
                charucoIds=charuco_ids)
       
        # If our image size is unknown, set it now
        if not image_size:
            image_size = gray.shape[::-1]
    
        # Reproportion the image, maxing width or height at 1000
        proportion = max(img.shape) / 1000.0
        img = cv2.resize(img, (int(img.shape[1]/proportion), int(img.shape[0]/proportion)))
        # Pause to display each image, waiting for key press
        cv2.imshow('Charuco board', img)
        cv2.waitKey(0)
    else:
        print("Not able to detect a charuco board in image: {}".format(iname))

# Destroy any open CV windows
cv2.destroyAllWindows()


# Make sure at least one image was found
if len(images) < 1:
    # Calibration failed because there were no images, warn the user
    print("Calibration was unsuccessful. No images of charucoboards were found. Add images of charucoboards and use or alter the naming conventions used in this file.")
    # Exit for failure
    exit()

# Make sure we were able to calibrate on at least one charucoboard by checking
# if we ever determined the image size
if not image_size:
    # Calibration failed because we didn't see any charucoboards of the PatternSize used
    print("Calibration was unsuccessful. We couldn't detect charucoboards in any of the images supplied. Try changing the patternSize passed into Charucoboard_create(), or try different pictures of charucoboards.")
    # Exit for failure
    exit()


calibration, cameraMatrix, distCoeffs, rvecs, tvecs = aruco.calibrateCameraCharuco(
        charucoCorners=corners_all,
        charucoIds=ids_all,
        board=CHARUCO_BOARD,
        imageSize=image_size,
        cameraMatrix=None,
        distCoeffs=None)

# Print matrix and distortion coefficient to the console
print(cameraMatrix)
print(distCoeffs)


[[1.73674041e+03 0.00000000e+00 6.42423055e+02]
 [0.00000000e+00 1.73591242e+03 4.84957603e+02]
 [0.00000000e+00 0.00000000e+00 1.00000000e+00]]
[[-0.21643089  0.17193363  0.00062022  0.00069881  0.0741379 ]]


In [12]:
h, w = img.shape[:2] # image size
newcameramtx, roi = cv.getOptimalNewCameraMatrix(cameraMatrix, distCoeffs, (w,h), 1, (w,h))
# read image and convert into gray scale
img = cv.imread(images[1])
# undistort
dst = cv.undistort(img, cameraMatrix, distCoeffs, None, newcameramtx)
# crop the image
x, y, w, h = roi
dst = dst[y:y+h, x:x+w]
# show undistorted image
cv.imshow('undistorted image', dst)
cv.imshow('img_distorted',img)
cv.waitKey(0)
cv.destroyAllWindows()
# save image
cv.imwrite('undistorted.png', dst)

True

# Détection des cercles sur les images avec MSER

In [16]:
import cv2 as cv2
import numpy as np


#------------------------------------------------#
#------------- 1. Read image ---------------#
#------------------------------------------------#
## 1.1 Read image as gray image
img = cv2.imread('./B1/GM8_B1/Basler_acA1300-200um__23048060__20240118_170811435_1560.tiff',cv2.IMREAD_GRAYSCALE)

#------------------------------------------------#
#------------- 2. MSER detection ---------#
#------------------------------------------------#
## 2.1 Create MSER object
mser = cv2.MSER_create()
## 2.2 Detect MSER regions
regions,_ = mser.detectRegions(img)

In [17]:
## 2.3 Draw MSER regions
# cv2.convexHull finds the convex hull of a 2D point set
# save the list of numpy array of contours a variable ‘hulls’
hulls = [cv2.convexHull(p.reshape(-1, 1, 2)) for p in regions]
# create a copy of the original image
clone = img.copy()
clone = cv2.cvtColor(img, cv2.COLOR_GRAY2RGB)
# draw contours on the image.
cv2.polylines(clone, hulls, 1, (0, 255, 0))
# show and save the image
cv2.namedWindow('all MSER', cv2.WINDOW_NORMAL)
cv2.imshow('all MSER', clone) # show the image
cv2.imwrite('all MSER.jpg',clone) # save the image
cv2.waitKey()
cv2.destroyAllWindows()

In [18]:
def circularity_check(img, regions, hulls):

 ## Circularity Check

 # create two empty lists to store interesting regions
     Regions = []
     Contours = []

 # iterate over regions and contours
 # zip() returns an iterator of tuples based on the iterable objects
 # enumerate() method adds a counter to an iterable and
 # returns it in a form of enumerating object
     for index, (region, cnt) in enumerate(zip(regions, hulls)):
         area = cv2. contourArea(cnt) # computes the area of the contours
         perimeter = cv2.arcLength(cnt,True) # computes contours perimeter
         circularity = 4*np.pi*area/perimeter**2 # circularity index
         if circularity > 0.93:
            if(len(cnt) < 14):
                Regions.append(region) # interesting regions
                Contours.append(cnt) # interesting contours

 ## Plot circular MSER

 # create a copy of the original image and convert it in gray image
     clone = img.copy()
     clone = cv2.cvtColor(img, cv2.COLOR_GRAY2RGB)

 # draw MSER
     cv2.polylines(clone, Contours, 1, (0, 255, 0))

     cv2.namedWindow('circular MSER', cv2.WINDOW_NORMAL)
     cv2.imshow('circular MSER', clone)
     cv2.imwrite('circular MSER.jpg',clone) # write the image
 # close the image
     cv2.waitKey()
     cv2.destroyAllWindows()

     return Regions, Contours, clone


In [19]:
Regions_circ, Contours_circ, clone_circ = circularity_check(img, regions, hulls)

In [20]:
def intensity_check(img, regions, hulls):
 ## Check on the pixel intensity

 # create three empty lists to store intereseting regions
 Regions = []
 Contours = []
 Centers = []

 # iterate over regions and contours
 for index, (region, cnt) in enumerate(zip(regions, hulls)):
     M = cv2.moments(cnt) # calculate the image moments
     center = int(M["m10"]/M["m00"]),int(M["m01"]/M["m00"]) # centroid (Cx, Cy)
     bright = img[center[1],center[0]] # brightness of the center (expected white)
     bright_mean = np.mean([img[p[1],p[0]] for p in region]) # mean brightness within a region
     if bright >= 0:
         Regions.append(region) # interesting regions
         Contours.append(cnt) # interesting contours
         Centers.append(center) # interesting centroids

 ## Plot selected MSER

 # create a copy of the original image
 clone = img.copy()
 clone = cv2.cvtColor(img, cv2.COLOR_GRAY2RGB)

 # draw and save MSER
 cv2.polylines(clone, Contours, 1, (0, 255, 0))


 cv2.namedWindow('selected MSER from intensity', cv2.WINDOW_NORMAL)
 cv2.imshow('selected MSER from intensity',clone)
 cv2.imwrite('selected MSER from intensity.jpg',clone)
 # close the image
 cv2.waitKey()
 cv2.destroyAllWindows()

 return Regions, Contours, Centers, clone

In [21]:
Regions_int, Contours_int, Centers_int, clone_int = intensity_check(img, Regions_circ, Contours_circ)

In [22]:
def duplicated_check(img, regions, cnt, centers):
 Box = []
 Box = [cv2.boundingRect(i) for i in cnt] # bounding box
 B = np.array(Box) # convert into array
 # create a list with 0 -> False in Boolean data type
 remove = np.zeros(B.shape[0], dtype=bool) # 0 is equal to False

 atol = 5 # tolerance

 # iterate over all boxes
 for i in range(B.shape[0]):

 # check if a box is similar to the others within a fixed
 # tolerance

 # np.isclose returns a boolean array (True or False)
 # where two arrays are element-wise equal within a tolerance
     similar = np.isclose(B[i, :], B[(i + 1):, :], atol = atol)

 # check if a box is similar to another on both sides

 # np.all Test whether all array elements along a given axis
 # evaluate to True
     equals = np.all(similar, axis=1)

 # Logical OR: True if either of the operands is true
     remove[(i + 1):] = np.logical_or(remove[(i + 1):], equals)

 # np.where return elements chosen from x or y depending on condition
 # Logical NOT: True if operand is false
 index = np.where(np.logical_not(remove))
 Bbox = B[np.logical_not(remove)]
 # create three empty lists to store intereseting regions
 Regions = []
 Contours = []
 Centers = []
 Contours = [cnt[p] for p in index[0]]
 Regions = [regions[p] for p in index[0]]
 Centers = [centers[p] for p in index[0]]

 ## Plot selected MSER

 # create a copy of the original image
 clone = img.copy()
 clone = cv2.cvtColor(img, cv2.COLOR_GRAY2RGB)

 # draw MSER
 cv2.polylines(clone, Contours, 1, (0, 255, 0))
 cv2.namedWindow('duplicated MSER suppression', cv2.WINDOW_NORMAL)
 cv2.imshow('duplicated MSER suppression',clone)
 cv2.imwrite('duplicated MSER suppression.jpg',clone)
 # close the image
 cv2.waitKey()
 cv2.destroyAllWindows()

 return Regions, Contours, Centers, Bbox, clone


In [23]:
Regions, Contours, Centers, Bbox, clone_sel = duplicated_check(img,Regions_int,Contours_int,Centers_int)

In [24]:
## 3.4 Evaluate mean centroid
result_centroid = []
temp = []
for ind in Bbox:
 temp = []
 corner_min = np.array((ind[0],ind[1]))
 corner_max = np.array((ind[0]+ind[2],ind[1]+ind[3]))

 for centroid in Centers:
     if ((np.array(centroid)<corner_max).all() and (np.array(centroid)>corner_min).all()):
         temp.append(centroid)
 result_centroid.append(tuple(np.mean(temp,axis=0,dtype = int)))
result_centroid = list(set(result_centroid))
print(result_centroid)

[(539, 727), (1137, 680), (614, 626), (704, 718), (474, 381), (620, 723), (929, 709), (36, 364), (30, 435), (741, 439), (933, 432), (885, 481), (847, 611), (860, 362), (531, 632), (691, 620), (1022, 510), (755, 525), (1196, 554), (678, 529), (1070, 424), (1138, 352), (1065, 686), (590, 447), (784, 713), (815, 436), (830, 521), (505, 741), (482, 502), (1093, 507), (863, 707), (600, 534), (14, 759), (522, 540), (27, 512), (1171, 466), (1232, 683), (990, 692), (666, 444), (969, 599), (770, 618), (24, 590), (1001, 427), (952, 514), (512, 452), (492, 598), (903, 573), (1042, 596), (18, 673)]


In [25]:
result_centroid = sorted(result_centroid, key=lambda x: x[1], reverse=True)
print(result_centroid)

[(14, 759), (505, 741), (539, 727), (620, 723), (704, 718), (784, 713), (929, 709), (863, 707), (990, 692), (1065, 686), (1232, 683), (1137, 680), (18, 673), (531, 632), (614, 626), (691, 620), (770, 618), (847, 611), (969, 599), (492, 598), (1042, 596), (24, 590), (903, 573), (1196, 554), (522, 540), (600, 534), (678, 529), (755, 525), (830, 521), (952, 514), (27, 512), (1022, 510), (1093, 507), (482, 502), (885, 481), (1171, 466), (512, 452), (590, 447), (666, 444), (741, 439), (815, 436), (30, 435), (933, 432), (1001, 427), (1070, 424), (474, 381), (36, 364), (860, 362), (1138, 352)]


In [26]:
#-----------------------------------------------------#
#------------- 4. Results visualization ---------#
#-----------------------------------------------------#
## 4.1 Label selected MSER
# create a copy of the image and convert it in gray image
clone1 = img.copy()
clone1 = cv2.cvtColor(img, cv2.COLOR_GRAY2RGB)
## set text font
font = cv2.FONT_HERSHEY_SIMPLEX
fontScale = 1
color = (0, 0, 255)
thickness = 2
nombre=0
## put number on the image
for i,c in enumerate(result_centroid):
 nombre+=1
 number = str(i+1)
 cv2.putText(clone1,number,c,font,fontScale,color,thickness)
## 4.2 Draw contours
print (nombre)
cv2.polylines(clone1, Contours, 1, (0, 255, 0))
cv2.namedWindow('selected MSER', cv2.WINDOW_NORMAL)
cv2.imshow('selected MSER',clone1) # show the image
cv2.imwrite('selected MSER.jpg',clone1) # save the image
# close the image
cv2.waitKey()
cv2.destroyAllWindows()

49
